In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import factorial
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Load the cleaned data
surgery = pd.read_csv('surgery_data_cleaned_for_modeling.csv')
provider_stats = pd.read_csv('provider_statistics_mmc_model.csv')

In [11]:
# Real provider theatre capacity and operating schedules
PROVIDER_SCHEDULES = {
    'CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST': {
        'theatres': 2, 
        'operating_hours_per_day': 10,  # 8am-6pm
        'days_per_week': 5,  # Monday-Friday + some Saturday sessions
        'sessions_per_week': 12  # Morning/afternoon sessions
    },
    'NORTH WEST ANGLIA NHS FOUNDATION TRUST': {
        'theatres': 4, 
        'operating_hours_per_day': 9,  # 8am-5pm
        'days_per_week': 5,
        'sessions_per_week': 20  # 4 theatres × 2 sessions × 2.5 days avg
    },
    'FITZWILLIAM HOSPITAL': {
        'theatres': 5, 
        'operating_hours_per_day': 10,  # 8am-6pm
        'days_per_week': 6,  # Monday-Saturday
        'sessions_per_week': 30  # More flexible independent sector scheduling
    },
    'ANGLIA COMMUNITY EYE SERVICE LTD': {
        'theatres': 3, 
        'operating_hours_per_day': 8,  # 9am-5pm
        'days_per_week': 5,
        'sessions_per_week': 15
    },
    'SPAMEDICA PETERBOROUGH': {
        'theatres': 2, 
        'operating_hours_per_day': 9,  # 8am-5pm
        'days_per_week': 6,  # Dedicated facility - more days
        'sessions_per_week': 24
    },
    'SPAMEDICA BEDFORD': {
        'theatres': 2, 
        'operating_hours_per_day': 9,
        'days_per_week': 5,
        'sessions_per_week': 20
    }
}

In [13]:
THEATRE_TIMES = {
    # Pre-operative preparation
    'patient_prep_time': 15,  # Patient positioning, draping, local anesthetic
    'equipment_setup': 5,     # Microscope, phaco machine setup
    
    # Operative times by HRG (from your document)
    'operative_times': {
        'BZ33Z': {'median': 7.5, 'std': 2.5, 'complexity': 'Minor'},
        'BZ32B': {'median': 12.5, 'std': 2.5, 'complexity': 'Intermediate 0-1'},
        'BZ32A': {'median': 17.5, 'std': 2.5, 'complexity': 'Intermediate 2+'},
        'BZ34C': {'median': 12.6, 'std': 4.6, 'complexity': 'Phaco 0-1'},  # From your document: consultants 19±10, but median ~12.6
        'BZ34B': {'median': 15, 'std': 5, 'complexity': 'Phaco 2-3'},
        'BZ34A': {'median': 18, 'std': 6, 'complexity': 'Phaco 4+'},
        'BZ30A': {'median': 25, 'std': 8, 'complexity': 'Complex 2+'},
        'BZ31B': {'median': 35, 'std': 10, 'complexity': 'Very Major 0-1'},
        'BZ31A': {'median': 50, 'std': 15, 'complexity': 'Very Major 2+'}
    },
    
    # Post-operative and turnover
    'patient_recovery_handover': 10,  # Brief recovery, instructions, discharge
    'theatre_cleaning': 15,           # Theatre cleaning between cases
    'equipment_changeover': 5,       # Equipment reset for next case
    
    # Session breaks
    'coffee_break': 15,              # Mid-morning/afternoon break
    'lunch_break': 45,               # Lunch break
    'end_of_session_cleanup': 20     # End of session cleanup
}


In [14]:
def calculate_realistic_theatre_capacity(provider_schedule, case_mix_distribution):
    """Calculate realistic daily theatre capacity accounting for all time components"""
    
    # Get schedule parameters
    theatres = provider_schedule['theatres']
    operating_hours = provider_schedule['operating_hours_per_day']
    sessions_per_week = provider_schedule['sessions_per_week']
    
    # Convert to daily sessions (assuming some variation in scheduling)
    avg_sessions_per_day = sessions_per_week / 5  # Spread over weekdays
    
    # Calculate time per case including all components
    case_mix_times = []
    for hrg, proportion in case_mix_distribution.items():
        if hrg in THEATRE_TIMES['operative_times']:
            # Total time per case = prep + operative + recovery + cleaning + changeover
            operative_time = THEATRE_TIMES['operative_times'][hrg]['median']
            total_case_time = (
                THEATRE_TIMES['patient_prep_time'] +
                THEATRE_TIMES['equipment_setup'] +
                operative_time +
                THEATRE_TIMES['patient_recovery_handover'] +
                THEATRE_TIMES['theatre_cleaning'] +
                THEATRE_TIMES['equipment_changeover']
            )
            case_mix_times.append(total_case_time * proportion)
    
    # Weighted average time per case
    avg_time_per_case = sum(case_mix_times)
    
    # Available time per session (accounting for breaks)
    session_length_minutes = (operating_hours * 60) / 2  # Assuming 2 sessions per day
    effective_session_time = session_length_minutes - THEATRE_TIMES['coffee_break'] - (THEATRE_TIMES['lunch_break'] / 2)
    
    # Cases per session per theatre
    cases_per_session = max(1, int(effective_session_time / avg_time_per_case))
    
    # Total daily capacity
    daily_capacity = theatres * avg_sessions_per_day * cases_per_session / theatres  # Per theatre
    weekly_capacity = daily_capacity * 5  # 5 working days
    
    # Effective service rate (patients per day per theatre)
    service_rate_per_theatre = daily_capacity
    
    return {
        'avg_time_per_case_minutes': avg_time_per_case,
        'cases_per_session': cases_per_session,
        'daily_capacity_per_theatre': daily_capacity,
        'weekly_capacity_total': weekly_capacity * theatres,
        'service_rate_per_theatre': service_rate_per_theatre,
        'effective_operating_hours': effective_session_time / 60 * 2  # Back to hours per day
    }

In [22]:
class RealisticMMcQueue:
    """M/M/c model with realistic theatre scheduling and turnover times"""
    
    def __init__(self, arrival_rate, provider_schedule, case_mix_distribution, name="Provider"):
        self.arrival_rate = arrival_rate  # patients/day
        self.provider_schedule = provider_schedule
        self.case_mix_distribution = case_mix_distribution
        self.name = name
        
        # Calculate realistic capacity
        capacity_analysis = calculate_realistic_theatre_capacity(provider_schedule, case_mix_distribution)
        
        self.theatres = provider_schedule['theatres']
        self.service_rate_per_theatre = capacity_analysis['service_rate_per_theatre']
        self.total_service_rate = self.service_rate_per_theatre * self.theatres
        self.avg_case_time_minutes = capacity_analysis['avg_time_per_case_minutes']
        self.cases_per_session = capacity_analysis['cases_per_session']
        
        # Traffic intensity and utilization
        self.rho = arrival_rate / self.service_rate_per_theatre
        self.utilization = arrival_rate / self.total_service_rate
        
        # Store capacity analysis for reporting
        self.capacity_analysis = capacity_analysis
        
    def calculate_metrics(self):
        """Calculate M/M/c metrics with realistic scheduling constraints"""
        
        # Check system stability
        if self.utilization >= 1.0:
            return {
                'stable': False,
                'utilization': self.utilization,
                'avg_wait_time_days': float('inf'),
                'avg_queue_length': float('inf'),
                'daily_capacity': self.total_service_rate,
                'capacity_analysis': self.capacity_analysis
            }
        
        # For high utilization systems, use approximation
        if self.utilization > 0.9:
            # Kingman's approximation for heavily loaded systems
            ca2 = 1  # Coefficient of variation squared for arrivals (Poisson = 1)
            cs2 = 1  # Coefficient of variation squared for service (Exponential = 1)
            
            avg_wait_time = (ca2 + cs2) / 2 * (self.utilization / (1 - self.utilization)) * (1 / self.total_service_rate)
        else:
            # Standard M/M/c calculation for stable systems
            c = self.theatres
            rho = self.rho
            
            try:
                # Calculate P0 (probability of empty system)
                sum_part = sum((rho**n) / factorial(min(n, 20)) for n in range(c))  # Limit factorial to avoid overflow
                
                if rho < c:
                    p0_denominator = sum_part + (rho**c / factorial(min(c, 20))) * (1 / (1 - rho/c))
                    p0 = 1 / p0_denominator if p0_denominator > 0 else 0
                else:
                    p0 = 0
                
                if p0 > 0 and rho < c:
                    # Expected queue length
                    lq = (rho**(c+1) * p0) / (factorial(min(c, 20)) * c * (1 - rho/c)**2)
                    # Expected waiting time
                    avg_wait_time = lq / self.arrival_rate if self.arrival_rate > 0 else 0
                else:
                    avg_wait_time = float('inf')
                    
            except (OverflowError, ZeroDivisionError):
                # Fallback to approximation
                avg_wait_time = (self.utilization / (1 - self.utilization)) * (1 / self.total_service_rate)
        
        return {
            'stable': self.utilization < 1.0,
            'utilization': self.utilization,
            'avg_wait_time_days': max(0, avg_wait_time),
            'avg_queue_length': max(0, avg_wait_time * self.arrival_rate),
            'daily_capacity': self.total_service_rate,
            'cases_per_session': self.cases_per_session,
            'avg_case_time_minutes': self.avg_case_time_minutes,
            'capacity_analysis': self.capacity_analysis
        }
        
    def calculate_realistic_metrics(self):
        """Calculate M/M/c metrics with realistic parameters"""
        if self.rho >= self.num_theatres or self.arrival_rate >= self.total_service_rate:
            return {
                'stable': False,
                'utilization': self.utilization,
                'avg_wait_time': float('inf'),
                'avg_queue_length': float('inf'),
                'throughput': min(self.arrival_rate, self.total_service_rate)
            }
        
        # Simplified M/M/c approximation for high utilization systems
        if self.utilization > 0.8:
            # Use approximation for heavily loaded systems
            waiting_time = (self.utilization ** (2 * self.num_theatres)) / (
                self.service_rate_per_theatre * (1 - self.utilization) * self.num_theatres)
        else:
            # Standard M/M/c formula
            c = self.num_theatres
            rho = self.rho
            
            # P0 calculation with numerical stability
            sum_part = sum((rho**n) / factorial(n) for n in range(c))
            if rho < c:
                p0 = 1 / (sum_part + (rho**c / factorial(c)) * (1 / (1 - rho/c)))
            else:
                p0 = 0
                
            if p0 > 0:
                lq = (rho**c * p0) / (factorial(c) * (1 - rho/c)**2)
                waiting_time = lq / self.arrival_rate
            else:
                waiting_time = float('inf')
        
        return {
            'stable': True,
            'utilization': self.utilization,
            'avg_wait_time': max(0, waiting_time),
            'avg_queue_length': max(0, waiting_time * self.arrival_rate),
            'throughput': self.arrival_rate
        }

In [16]:
def analyze_current_system():
    """Analyze current fragmented system with real data"""
    print("\n=== CURRENT FRAGMENTED SYSTEM ANALYSIS ===")
    
    results = []
    
    # Analyze each major provider
    for provider_name in PROVIDER_CAPACITY.keys():
        provider_data = surgery[surgery['Provider_Clean'] == provider_name.upper()]
        
        if len(provider_data) == 0:
            continue
            
        # Calculate arrival rate (procedures per day)
        date_range = (provider_data['End_clock_date'].max() - 
                     provider_data['Start_clock_date'].min()).days
        arrival_rate = len(provider_data) / max(date_range, 365)
        
        # Get operating time distributions for this provider's case mix
        hrg_distribution = provider_data['HRG'].value_counts(normalize=True)
        operating_times = {hrg: OPERATING_TIMES.get(hrg, OPERATING_TIMES['BZ34C']) 
                          for hrg in hrg_distribution.index}
        
        # Get theatre capacity
        theatre_count = PROVIDER_CAPACITY[provider_name]
        
        # Create enhanced M/M/c model
        provider_queue = EnhancedMMcQueue(
            arrival_rate=arrival_rate,
            operating_times_dist=operating_times,
            num_theatres=theatre_count,
            name=provider_name
        )
        
        metrics = provider_queue.calculate_realistic_metrics()
        
        results.append({
            'Provider': provider_name[:30],
            'Procedures': len(provider_data),
            'Arrival_Rate': arrival_rate,
            'Theatres': theatre_count,
            'Utilization': metrics['utilization'],
            'Wait_Time_Days': metrics['avg_wait_time'],
            'Queue_Length': metrics['avg_queue_length'],
            'Stable': metrics['stable'],
            'Throughput': metrics['throughput']
        })
    
    return pd.DataFrame(results)

In [17]:
def analyze_consolidated_system(current_results):
    """Analyze consolidated system with pooled resources"""
    print("\n=== CONSOLIDATED SYSTEM ANALYSIS ===")
    
    # Aggregate system parameters
    total_procedures = current_results['Procedures'].sum()
    total_arrival_rate = current_results['Arrival_Rate'].sum()
    total_theatres = current_results['Theatres'].sum()
    
    # Weighted operating time distribution across all providers
    surgery_sample = surgery[surgery['Provider_Clean'].isin([
        p.upper() for p in PROVIDER_CAPACITY.keys()])]
    
    hrg_distribution = surgery_sample['HRG'].value_counts(normalize=True)
    consolidated_operating_times = {hrg: OPERATING_TIMES.get(hrg, OPERATING_TIMES['BZ34C']) 
                                   for hrg in hrg_distribution.index}
    
    # Create consolidated M/M/c model
    consolidated_queue = EnhancedMMcQueue(
        arrival_rate=total_arrival_rate,
        operating_times_dist=consolidated_operating_times,
        num_theatres=total_theatres,
        name="Consolidated System"
    )
    
    consolidated_metrics = consolidated_queue.calculate_realistic_metrics()
    
    # Current system weighted averages
    current_avg_wait = (current_results['Wait_Time_Days'] * current_results['Procedures']).sum() / total_procedures
    current_avg_utilization = (current_results['Utilization'] * current_results['Theatres']).sum() / total_theatres
    
    print(f"Current Fragmented System:")
    print(f"  Total Procedures: {total_procedures}")
    print(f"  Total Arrival Rate: {total_arrival_rate:.2f} patients/day")
    print(f"  Total Theatres: {total_theatres}")
    print(f"  Weighted Avg Utilization: {current_avg_utilization:.3f}")
    print(f"  Weighted Avg Wait Time: {current_avg_wait:.2f} days")
    
    print(f"\nConsolidated System:")
    print(f"  System Utilization: {consolidated_metrics['utilization']:.3f}")
    print(f"  Expected Wait Time: {consolidated_metrics['avg_wait_time']:.2f} days")
    print(f"  Expected Queue Length: {consolidated_metrics['avg_queue_length']:.1f} patients")
    print(f"  System Stable: {consolidated_metrics['stable']}")
    
    # Calculate improvements
    if current_avg_wait > 0 and consolidated_metrics['avg_wait_time'] != float('inf'):
        wait_improvement = (current_avg_wait - consolidated_metrics['avg_wait_time']) / current_avg_wait * 100
        utilization_improvement = (consolidated_metrics['utilization'] - current_avg_utilization) / current_avg_utilization * 100
        
        print(f"\n🎯 CONSOLIDATION BENEFITS:")
        print(f"  Wait Time Reduction: {wait_improvement:.1f}%")
        print(f"  Utilization Improvement: {utilization_improvement:.1f}%") 
        print(f"  Resource Pooling Effect: Reduced variability through shared capacity")
        
        return {
            'wait_improvement': wait_improvement,
            'utilization_improvement': utilization_improvement,
            'consolidated_metrics': consolidated_metrics
        }
    
    return {'consolidated_metrics': consolidated_metrics}

In [18]:
def capacity_scenario_analysis(base_results):
    """Analyze different capacity scenarios"""
    print("\n=== CAPACITY SCENARIO ANALYSIS ===")
    
    total_arrival_rate = base_results['Arrival_Rate'].sum()
    base_theatres = base_results['Theatres'].sum()
    
    scenarios = {
        'Current': base_theatres,
        'Hinchingbrooke Expansion (+3)': base_theatres + 3,  # New 7-theatre block
        'System Optimization (+2)': base_theatres + 2,
        'Full Capacity (+5)': base_theatres + 5
    }
    
    surgery_sample = surgery[surgery['Provider_Clean'].isin([
        p.upper() for p in PROVIDER_CAPACITY.keys()])]
    hrg_distribution = surgery_sample['HRG'].value_counts(normalize=True)
    operating_times = {hrg: OPERATING_TIMES.get(hrg, OPERATING_TIMES['BZ34C']) 
                      for hrg in hrg_distribution.index}
    
    scenario_results = []
    
    for scenario_name, theatre_count in scenarios.items():
        queue = EnhancedMMcQueue(
            arrival_rate=total_arrival_rate,
            operating_times_dist=operating_times,
            num_theatres=theatre_count,
            name=scenario_name
        )
        
        metrics = queue.calculate_realistic_metrics()
        
        scenario_results.append({
            'Scenario': scenario_name,
            'Theatres': theatre_count,
            'Utilization': metrics['utilization'],
            'Wait_Time': metrics['avg_wait_time'],
            'Stable': metrics['stable']
        })
    
    scenario_df = pd.DataFrame(scenario_results)
    print(scenario_df.round(3))
    
    return scenario_df

In [19]:
print("Loading data and running enhanced M/M/c analysis...")

# Analyze current fragmented system
current_system_results = analyze_current_system()
print("\nCURRENT SYSTEM PERFORMANCE:")
print(current_system_results.round(3))


Loading data and running enhanced M/M/c analysis...

=== CURRENT FRAGMENTED SYSTEM ANALYSIS ===


NameError: name 'PROVIDER_CAPACITY' is not defined

In [ ]:
# Analyze consolidated system
consolidation_results = analyze_consolidated_system(current_system_results)

# Capacity scenario analysis
scenario_results = capacity_scenario_analysis(current_system_results)

# Visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Provider utilization vs capacity
ax1.scatter(current_system_results['Theatres'], current_system_results['Utilization'], 
           s=current_system_results['Procedures']/10, alpha=0.7)
ax1.set_xlabel('Number of Theatres')
ax1.set_ylabel('Utilization Rate')
ax1.set_title('Theatre Utilization vs Capacity\n(Bubble size = Procedure Volume)')
ax1.axhline(y=0.8, color='red', linestyle='--', label='80% Target')
ax1.legend()

# 2. Wait times by provider
providers_short = [p[:15] for p in current_system_results['Provider']]
ax2.bar(providers_short, current_system_results['Wait_Time_Days'], color='skyblue')
ax2.set_xlabel('Provider')
ax2.set_ylabel('Average Wait Time (days)')
ax2.set_title('Wait Times by Provider')
ax2.tick_params(axis='x', rotation=45)

# 3. Scenario analysis
ax3.bar(scenario_results['Scenario'], scenario_results['Wait_Time'], 
        color=['red', 'orange', 'yellow', 'green'])
ax3.set_xlabel('Capacity Scenario')
ax3.set_ylabel('Wait Time (days)')
ax3.set_title('Wait Times Under Different Capacity Scenarios')
ax3.tick_params(axis='x', rotation=45)

# 4. Utilization vs Wait Time relationship
ax4.scatter(current_system_results['Utilization'], current_system_results['Wait_Time_Days'])
ax4.set_xlabel('System Utilization')
ax4.set_ylabel('Wait Time (days)')
ax4.set_title('Utilization vs Wait Time Relationship')

plt.tight_layout()
plt.show()